In [15]:
import torch
from torchvision import transforms, models
from PIL import Image
import joblib
import torch.nn.functional as F

# Load the label encoder
label_encoder = joblib.load('models/tcg_magic/labels.joblib')

# Get the number of classes from the label encoder
num_classes = len(label_encoder.classes_)

# Create the model with the correct architecture (assuming you used ResNet50 during training)
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Load the trained model
state_dict = torch.load('models/tcg_magic/classifier.pth', weights_only=True)

# If your model was trained with DataParallel, remove the 'module.' prefix
if 'module.' in next(iter(state_dict.keys())):
    state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}

model.load_state_dict(state_dict)
model.eval()  # Set the model to evaluation mode

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Preprocess the Image
Apply the same transformations that were used during training.

In [16]:
# Define the transformation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load and preprocess the image
image_path = 'datasets/tcg_magic/data/test/00c833d3_d406_4c24_a4ab_9302fc27497e/0004.png'
image = Image.open(image_path).convert('RGB')
image = transform(image).unsqueeze(0)  # Add batch dimension

## Predict the Label


In [17]:
import torch.nn.functional as F

# Make prediction
with torch.no_grad():
    outputs = model(image)
    probabilities = F.softmax(outputs, dim=1)
    confidence, predicted = torch.max(probabilities, 1)

# Get the predicted class name and confidence
predicted_class = label_encoder.inverse_transform([predicted.item()])[0]
confidence_percentage = confidence.item() * 100

print(f"Predicted class: {predicted_class}")
print(f"Confidence: {confidence_percentage:.2f}%")

Predicted class: 00c833d3_d406_4c24_a4ab_9302fc27497e
Confidence: 95.46%


In [18]:
# Make prediction
with torch.no_grad():
    outputs = model(image)
    probabilities = F.softmax(outputs, dim=1)

# Get top 5 predictions
top5_prob, top5_catid = torch.topk(probabilities, 5)
top5_prob = top5_prob.squeeze().tolist()
top5_catid = top5_catid.squeeze().tolist()

# Print top 5 predictions
print("Top 5 predictions:")
for i in range(5):
    class_name = label_encoder.inverse_transform([top5_catid[i]])[0]
    confidence_percentage = top5_prob[i] * 100
    print(f"{i+1}. {class_name}: {confidence_percentage:.2f}%")

Top 5 predictions:
1. 00c833d3_d406_4c24_a4ab_9302fc27497e: 95.46%
2. 006b5880_eb49_4198_930c_39a85ff6ec66: 0.40%
3. 00464232_2594_405a_95bb_f9b2a3287871: 0.36%
4. 00afdd58_a6e3_490c_8ff5_3d761fcd3885: 0.20%
5. 008d7d1f_8635_41d5_9921_11d8043a6206: 0.16%
